In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

# Set up the WebDriver (e.g., Chrome)
# Make sure to provide the path to your WebDriver executable.
driver = webdriver.Chrome()

# Navigate to a website
url = 'https://example.com'  # Replace with your target website
driver.get(url)

In [51]:
seats = pd.read_excel('CED_2021_AUST.xlsx')
seats = seats[['CED_CODE_2021', 'CED_NAME_2021', 'STATE_NAME_2021']].drop_duplicates()
seats = seats.rename(columns={'CED_CODE_2021':'quickstats_code'})

In [17]:
seat_codes = seats['quickstats_code'].to_list()

df = pd.DataFrame()

for seat_code in seat_codes:
    try:
        url = 'https://www.abs.gov.au/census/find-census-data/quickstats/2021/CED' + seat_code
        driver.get(url)
        time.sleep(5)
        temp = {
            'seat_code': seat_code,
            'indig': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td[2]').text),
            'age': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[2]/div[2]/div[3]/table/tbody/tr[2]/td[1]').text),
            't_ed': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td[2]').text),
            'income': float(driver.find_element(By.XPATH, r'/html/body/div[1]/main/div[3]/div/div/div[3]/div[1]/div[5]/div[2]/div[9]/table/tbody/tr[2]/td[1]').text[1:].replace(',', ''))
        }
        df = pd.concat(
            [df, pd.DataFrame(temp, index=[0])], ignore_index=True
        )
    except:
        pass

In [19]:
driver.quit()

In [52]:

# Replace 'seat_code' and 'quickstats_code' with your actual column names
result_df = df.merge(seats, how='left', left_on='seat_code', right_on='quickstats_code').drop(columns=['seat_code'])

# The result_df will contain the left join of df and seats based on the specified columns


In [53]:
state_acronyms = {
    'New South Wales': 'NSW',
    'Victoria': 'Vic',
    'Queensland': 'Qld',
    'South Australia': 'SA',
    'Australian Capital Territory': 'ACT',
    'Northern Territory': 'NT',
    'Tasmania': 'Tas',
    'Western Australia': 'WA'
}

# Update the 'State_acronym' column based on the dictionary
result_df['State_acronym'] = result_df['STATE_NAME_2021'].map(state_acronyms)

In [34]:
# Set up the WebDriver (e.g., Chrome)
# Make sure to provide the path to your WebDriver executable.
driver = webdriver.Chrome()

# Navigate to a website
url = 'https://example.com'  # Replace with your target website
driver.get(url)

In [35]:
dem_rating = []

for i, row in result_df.iterrows():
    url = 'https://www.aec.gov.au/profiles/' + row['State_acronym'] + '/' + row['CED_NAME_2021'] + '.htm'
    driver.get(url)
    time.sleep(2)
    dem_rating.append(driver.find_element(By.XPATH, r'/html/body/div[3]/div/div/dl/dd[9]/strong').text)

In [37]:
driver.quit()

In [54]:
result_df['dem_rating'] = dem_rating

In [55]:
f = open('Hor.json')
data = f.read()
hor = json.loads(data)
hor = pd.DataFrame(hor)
f.close()
hor['electorate'] = hor['Representing'].str.split(',').str[0]
hor = hor[['electorate', 'FullName', 'Party']]
result_df = result_df.merge(hor, how='left', left_on='CED_NAME_2021', right_on='electorate')
result_df = result_df.drop(columns='electorate')

In [59]:
twopp = pd.read_csv('HouseTppByDivisionDownload-27966.csv', skiprows=1)
twopp = twopp[['DivisionNm', 'DivisionID', 'Australian Labor Party Percentage']]
result_df = result_df.merge(twopp, how='left', left_on='CED_NAME_2021', right_on='DivisionNm')
result_df = result_df.drop(columns='DivisionNm')

In [62]:
result_df['Coalition point lead'] = (result_df['Australian Labor Party Percentage'] - 50) * (-1)

In [63]:
result_df.to_csv('demographicData.csv', index=False)